In [1]:
%pip install git+https://github.com/rDanielNutt/Qsim.git
%pip install npy-append-array

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rDanielNutt/Qsim.git to /tmp/pip-req-build-owfz63z8
  Running command git clone --filter=blob:none --quiet https://github.com/rDanielNutt/Qsim.git /tmp/pip-req-build-owfz63z8
  Resolved https://github.com/rDanielNutt/Qsim.git to commit e26b6ba308cb853a0246d1663b8d0c53fcc9c662
  Preparing metadata (setup.py) ... done
  Created wheel for qsim: filename=qsim-0.0.1-py3-none-any.whl size=7092 sha256=e25167063e6bb4bc98ddefa2674f75e70f4c471cf776ca1b260809076c926681
  Stored in directory: /tmp/pip-ephem-wheel-cache-cwsj82gy/wheels/ae/e7/a5/494f414ec450b291077ca7696b8f9c1e649eb6d217434b880c
Successfully built qsim
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
from numpy.random import randint
import cupy as cp
from cupyx.scipy.signal import fftconvolve
from cupyx.scipy import fft
from npy_append_array import NpyAppendArray as npy

from qsim.models import SchroModel
from qsim.schrosim import SchroSim

import math
import os

# Activation Functions

In [4]:
def sigmoid(x):
    e = cp.exp(x)
    a = 1 / (1 + (1/e))
    da = e / cp.square(e + 1)
    return a, da

def tanh(x):
    a = cp.tanh(x)
    da = 1 / cp.square(cp.cosh(x))
    return a, da

def softmax(x):
    e = cp.exp(x)
    esum = cp.sum(e, axis=1, keepdims=True)
    a = e / esum
    da = ((esum - e) * e) / cp.square(esum)
    return a, da

def relu(x):
    da = cp.where(x > 0, 1, 0)
    a = x * da
    return a, da

def comp_norm(x):
    asum = cp.sqrt(cp.nansum(cp.square(cp.abs(x)), axis=(1,2,3), keepdims=True) * 1e-4)
    a = x / asum
    da = ((1+1j) / asum) - (((1e-4 + 1e-4j) * cp.square(x)) / cp.square(asum))
    return a, da

def none(x):
    return x, 1

activation_funcs = {
    'sigmoid': sigmoid,
    'tanh': tanh,
    'softmax': softmax,
    'relu': relu,
    'norm': comp_norm,
    'none': none,
}


# Loss Functions

In [5]:
def mse(true, pred):
    loss = cp.mean(cp.abs(pred - true))
    dloss = pred - true
    return loss, dloss

def cat_crossentropy(true, pred):
    loss = cp.mean(-(true * cp.log(pred)) + ((1 - true) * cp.log(1 - pred)))
    dloss = - (pred - true) / (cp.square(pred) - pred)
    return loss, dloss

def wave_loss(true, pred):
    loss = cp.mean(cp.abs(true) * cp.abs(pred - true))
    dloss = cp.abs(true) * (pred - true)
    return loss, dloss

loss_funcs = {
    'mse': mse,
    'catcrossentropy': cat_crossentropy,
    'wave': wave_loss,
}

# Layer Classes

In [22]:

class BaseConv2D:
    
    def __init__(self, n_kernels, kernel_size, padding='valid', padding_mode='constant', activation='sigmoid', input_size=(), lr=1e-2, dtype=cp.float64, **kwargs):
        
        self.pad_type = padding
        self.pad_mode = padding_mode
        self.activate = activation_funcs[activation]
        self.lr = lr
        self.n_kernels = n_kernels

        if isinstance(input_size, int):
            self.input_size = (input_size, input_size, 1)
        elif isinstance(input_size, tuple):
            self.input_size = (*input_size, *([1]*(3-len(input_size))))

        if isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size, self.input_size[2])
        elif len(kernel_size) == 1:
            self.kernel_size = (kernel_size[0], 1, self.input_size[2])
        elif len(kernel_size) == 2:
            self.kernel_size = (*kernel_size, self.input_size[2])

        self.weights = cp.random.rand(1, *self.kernel_size, self.n_kernels).astype(dtype) / 100
        self.bias = cp.zeros([1, 1, 1, self.n_kernels]).astype(dtype)

        if self.pad_type == 'same':
            pre0 = math.ceil((self.kernel_size[0] - 1) / 2)
            post0 = math.floor((self.kernel_size[0] - 1) / 2)
            pre1 = math.ceil((self.kernel_size[1] - 1) / 2)
            post1 = math.floor((self.kernel_size[1] - 1) / 2)
        elif self.pad_type == 'full':
            pre0 = post0 = self.kernel_size[0] - 1
            pre1 = post1 = self.kernel_size[1] - 1
        else:
            pre0 = post0 = pre1 = post1 = 0

        self.pad = ((0,0), (pre0, post0), (pre1, post1), (0,0))
        self.output_size = (self.input_size[0] - self.kernel_size[0] + pre0+post0+1, self.input_size[1] - self.kernel_size[1] + pre1+post1+1, self.n_kernels)
        self.dact = cp.empty([])

    def forward(self, x):
        if self.pad_mode == 'wrap':
            self.x = cp.pad(x, self.pad, 'wrap')[:,:,:,:,cp.newaxis]
        else:
            self.x = cp.pad(x, self.pad, 'constant', constant_values=0)[:,:,:,:,cp.newaxis]
        act, self.dact = self.activate(cp.sum(fftconvolve(self.x, self.weights, mode='valid', axes=(1, 2)), axis=3) + self.bias)
        return act
    
    def backprop(self, grad):
        grad = (self.dact * grad)[:,:,:,cp.newaxis]
        grad = cp.where(cp.isfinite(grad), grad, 0)
        wgrad = cp.sum(fftconvolve(cp.flip(self.x, axis=(1,2)), grad, mode='valid', axes=(1, 2)), axis=0, keepdims=True)
        bgrad = cp.sum(grad, axis=(0, 1, 2))

        p0 = self.pad[1]
        p1 = self.pad[2]
        grad = cp.sum(fftconvolve(grad, cp.flip(self.weights, axis=(1, 2)), mode='full', axes=(1, 2)), axis=4)

        self.bias -= (self.lr * bgrad)
        self.weights -= (self.lr * wgrad)

        return grad[:, p0[0]:grad.shape[1]-p0[1], p1[0]:grad.shape[2]-p1[1], :]


class BaseFlatten:
    def __init__(self, input_size, **kwargs):
        self.input_size = input_size
        self.output_size = np.prod(input_size)
    
    def forward(self, x):
        return x.reshape([-1, self.output_size])

    def backprop(self, grad):
        return grad.reshape([-1, *self.input_size])


class BaseDense:
    def __init__(self, n_neurons, input_size, lr, activation, dtype):
        
        self.weights = cp.random.rand(input_size, n_neurons).astype(dtype) / 10
        self.bias = cp.zeros([1, n_neurons]).astype(dtype)
        
        self.lr = lr
        self.activate = activation_funcs[activation]
        self.output_size = n_neurons
        self.x = cp.empty([])
        self.dact = cp.empty([])

    def forward(self, x):
        self.x = x.copy()
        act, self.dact = self.activate(cp.dot(self.x, self.weights) + self.bias)
        return act
    
    def backprop(self, grad):
        grad = self.dact * grad
        grad = cp.where(cp.isfinite(grad), grad, 0)
        wgrad = cp.dot(self.x.T, grad)
        bgrad = cp.sum(grad, axis=0, keepdims=True)

        grad = cp.dot(grad, self.weights.T)

        self.weights -= (self.lr * wgrad)
        self.bias -= (self.lr * bgrad)

        return grad

def Flatten(input_size=()):
    class Flatten(BaseFlatten):
        def __init__(self, input_size=input_size, **kwargs):
            super().__init__(input_size, **kwargs)
    
    return Flatten

def Dense(n_neurons=0, input_size=(), lr=1e-2, activation='sigmoid'):
    class Dense(BaseDense):
        def __init__(self, n_neurons=n_neurons, input_size=input_size, lr=lr, activation=activation):
            super().__init__(n_neurons, input_size, lr, activation)
    
    return Dense

def Conv2D(n_kernels=0, kernel_size=0, padding='valid', padding_mode='constant', activation='sigmoid', input_size=()):
    class Conv2D(BaseConv2D):
        def __init__(self, n_kernels=n_kernels, kernel_size=kernel_size, padding=padding, padding_mode=padding_mode, activation=activation, input_size=input_size, **kwargs):
            super().__init__(n_kernels, kernel_size, padding, padding_mode, activation, input_size, **kwargs)

    return Conv2D

layer_funcs = {
    'Flatten': Flatten,
    'Dense': Dense,
    'Conv2D': Conv2D,
}

# Sequential Model Class

In [23]:
class Sequential:
    def __init__(self, layers=[], loss='mse', lr=1e-2, dtype=cp.float64):
        self.loss = loss_funcs[loss]
        self.lr = lr
        self.dtype = dtype

        if len(layers) > 0:
            self.layers = [layers[0](lr=self.lr, dtype=self.dtype)]
            output_size = self.layers[0].output_size

            for layer in layers[1:]:
                self.layers.append(layer(input_size=output_size, lr=self.lr, dtype=self.dtype))
                output_size = self.layers[-1].output_size
        else:
            self.layers = []

        self.history = []

    def save(self, path, name):
        if not os.path.exists(path):
            os.mkdir(path)

        if not os.path.exists(f'{path}{name}/'):
            os.mkdir(f'{path}{name}/')

        with npy(f'{path}{name}/history.npy', delete_if_exists=True) as h:
            h.append(np.array(self.history))

        for i, layer in enumerate(self.layers):
            l_type = str(type(layer).__name__)
            try:
                with npy(f'{path}{name}/{i}_{l_type}_weights.npy', delete_if_exists=True) as w:
                    w.append(layer.weights)
                with npy(f'{path}{name}/{i}_{l_type}_bias.npy', delete_if_exists=True) as b:
                    b.append(layer.bias)
            except AttributeError:
                continue

    def load_weights(self, path, name):
        for i, layer in enumerate(self.layers):
            l_type = str(type(layer).__name__)
            try:
                layer.weights = cp.load(f'{path}{name}/{i}_{l_type}_weights.npy')
                layer.bias = cp.load(f'{path}{name}/{i}_{l_type}_bias.npy')
            except AttributeError:
                continue

    def add(self, layer):
        if len(self.layers) > 0:
            output_size = self.layers[-1].output_size
            self.layers.append(layer(input_size=output_size, lr=self.lr, dtype=self.dtype))
        else:
            self.layers.append(layer(lr=self.lr, dtype=self.dtype))
    
    def set(self, loss=None, lr=None):
        if loss is not None:
            self.loss = loss_funcs[loss]
        if lr is not None:
            self.lr = lr

        for layer in self.layers:
            layer.lr = self.lr

    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backprop(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backprop(grad)

    def train_batch(self, x, y, epochs=1):
        for epoch in range(1, epochs+1):
            pred = self.predict(x)
            loss, grad = self.loss(y, pred)

            if not cp.all(cp.isfinite(loss)):
                raise Exception('Training Error: Non-finite values present in loss') 
            if not cp.all(cp.isfinite(grad)):
                raise Exception('Training Error: Non-finite values present in grad')

            self.backprop(grad)
            self.history.append(loss.get())
    

    def train(self, x, y, batch_size=None, epochs=1, loss=None, lr=None):
        self.set(loss, lr)

        if batch_size is not None:
            batches = [range(i, i+batch_size) for i in range(0, y.shape[0]-batch_size, batch_size)]
            batches += [range(y.shape[0]-(y.shape[0] % batch_size), y.shape[0])]*(y.shape[0] % batch_size > 0)
        else:
            batches = [range(y.shape[0])]

        for epoch in range(1, epochs+1):
            print(f'Epoch {epoch: >5}/{epochs: <3}:\n')

            for i, batch in enumerate(batches):
                pred = self.predict(cp.array(x[batch]))
                
                loss, grad = self.loss(cp.array(y[batch]), pred)
                self.backprop(grad)

            self.history.append(loss)
            print(f'loss[{loss:.3f}]\n')


# Creating Model Classes

In [31]:
class WaveModel(SchroModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.sim_steps = cp.empty([0, *self.model.layers[0].input_size[:-1], 2])

    def add_step(self, phi):
        step = cp.stack([cp.sum(phi, axis=0), cp.sum(self.sim.V + self.sim.ev, axis=0)], axis=-1)[cp.newaxis]
        self.sim_steps = cp.append(self.sim_steps, step, axis=0)
    
    def predict(self, phi, n_samp):
        self.sim.e_field(phi, n_samp)
        self.add_step(phi)

        pred = self.model.predict(self.sim_steps)[:, :, :, 0] 
        self.clear_steps()
        return self.sim.norm(pred)


class EVModel(SchroModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.sim_steps = cp.empty([0, *self.model.layers[0].input_size[:-1], 2])

    def add_step(self, phi):
        Vt = (cp.sum(self.sim.ev, axis=0, keepdims=True) - self.sim.ev) + self.sim.V
        phin = cp.exp(1j * Vt * self.sim.dt/2.0) * phi
        ev_step = cp.real(cp.log(cp.sum(phin, axis=0)/cp.sum(phi, axis=0)) * (-2j/self.sim.dt)) - self.sim.V
        ev_step = cp.where(cp.isfinite(ev_step), ev_step, 0)

        step = cp.stack([cp.sum(cp.square(cp.abs(phi)), axis=0, keepdims=True), ev_step*-1], axis=-1)
        self.sim_steps = cp.append(self.sim_steps, step, axis=0)
    
    def train(self, epochs):
        self.model.train_batch(self.sim_steps[:, :, :, :1], self.sim_steps[:, :, :, 1:], epochs=epochs)
        self.clear_steps()
        return np.mean(self.model.history[-epochs:])

    def predict(self, phi, n_samp):
        phi = cp.sum(phi, axis=0)

        ev = -1 * self.model.predict(cp.square(cp.abs(phi))[cp.newaxis, :, :, cp.newaxis])[:, :, :, 0]
        phi *= cp.exp(1j * (self.sim.V + ev) * self.sim.dt / 2.0)

        phihat = fft.fftn(phi)
        phihat = cp.exp(self.sim.dt * (-1j * fft.ifftshift(cp.square(self.sim.dists))/(2.0 * self.sim.me)))  * phihat
        phi = fft.ifftn(phihat)

        ev = -1 * self.model.predict(cp.square(cp.abs(phi))[cp.newaxis, :, :, cp.newaxis])[:, :, :, 0]
        phi *= cp.exp(1j * (self.sim.V + ev) * self.sim.dt / 2.0)
        
        self.sim.ev = ev
        return self.sim.norm(phi)

        

# Training Models

In [25]:
wave_mod = Sequential([
    Conv2D(n_kernels=1, kernel_size=3, padding='same', padding_mode='wrap', input_size=(500,500,2), activation='norm')
    ], loss='wave', lr=1e-7, dtype=cp.complex128)
#wave_mod.layers[0].weights *= 1j


for i in range(10):
    sim = SchroSim()
    sim.add_proton(pos=randint(-2, 3, 2))
    sim.add_electron(p=randint(-20, 21, 2), pos=randint(-2, 3, 2))

    mod = WaveModel(model=wave_mod, sim=sim)
    sim.simulate(dims=(5,5), dau=1e-2, time_step=1e-1, steps=5000, model=mod, train_model=20, ev_samp_rate=0)
    wave_mod.save(path='./wave1_checks/', name=f'wave1_{i+1}')
    print(f'Simulation wave training {i+1} done')


Model Trained at Step 20: loss [0.03982271657477765]
Model Trained at Step 40: loss [0.04017058619489774]
Model Trained at Step 60: loss [0.04019960483870215]
Model Trained at Step 80: loss [0.04021560787140915]
Model Trained at Step 100: loss [0.040166055137399057]
Model Trained at Step 120: loss [0.04015627994889745]
Model Trained at Step 140: loss [0.04016447894660521]
Model Trained at Step 160: loss [0.04016420432118802]
Model Trained at Step 180: loss [0.04022447245224293]
Model Trained at Step 200: loss [0.040643698893409776]
Model Trained at Step 220: loss [0.04072228399896503]
Model Trained at Step 240: loss [0.04083953586416002]
Model Trained at Step 260: loss [0.04082953302449214]
Model Trained at Step 280: loss [0.04077404443657022]
Model Trained at Step 300: loss [0.0407623814585762]
Model Trained at Step 320: loss [0.040718644804522613]
Model Trained at Step 340: loss [0.04069226406499905]
Model Trained at Step 360: loss [0.0406573988589312]
Model Trained at Step 380: loss

In [34]:
ev_mod = Sequential([Conv2D(1, kernel_size=11, padding='same', padding_mode='wrap', activation='relu', input_size=(500,500,1))],
                    loss='mse', lr=1e-7)

for i in range(10):
    sim = SchroSim()
    sim.add_proton(pos=randint(-2, 3, 2))
    sim.add_electron(p=randint(-20, 21, 2), pos=randint(-2, 3, 2))
    sim.add_electron(p=randint(-20, 21, 2), pos=randint(-2, 3, 2))

    mod = EVModel(model=ev_mod, sim=sim)
    sim.simulate(dims=(5,5), dau=1e-2, time_step=1e-1, steps=5000, model=mod, train_model=20, ev_samp_rate='full')
    ev_mod.save(path='./ev1_checks/', name=f'ev1_{i+1}')
    print(f'Simulation ev training {i+1} done')

    

Model Trained at Step 20: loss [3.9983947431408766]
Model Trained at Step 40: loss [2.745314880293292]
Model Trained at Step 60: loss [2.799250393596618]
Model Trained at Step 80: loss [2.7922966345275437]
Model Trained at Step 100: loss [2.786835999285194]
Model Trained at Step 120: loss [2.779938272861137]
Model Trained at Step 140: loss [2.765143389508612]
Model Trained at Step 160: loss [2.71893718776563]
Model Trained at Step 180: loss [2.682455635856931]
Model Trained at Step 200: loss [2.642248631482377]
Model Trained at Step 220: loss [2.6012314392497604]
Model Trained at Step 240: loss [2.571808219183864]
Model Trained at Step 260: loss [2.5406557582179503]
Model Trained at Step 280: loss [2.503498400301546]
Model Trained at Step 300: loss [2.453588776650002]
Model Trained at Step 320: loss [2.4043153217564552]
Model Trained at Step 340: loss [2.353846437463404]
Model Trained at Step 360: loss [2.3039906507417554]
Model Trained at Step 380: loss [2.245162548368134]
Model Train